In [1]:
import os
import json
import random
from glob import glob
import re
import pandas as pd
import gpt_2_simple as gpt2

qa_jokes_filepath = os.path.join('data', 'qa_jokes.csv')
small_jokes_filepath = os.path.join('data', 'small_jokes.csv')
transcripts_path = os.path.join('data', 'transcripts')

qa_jokes_prep_outpath = os.path.join('data', 'prep', 'qa_jokes_gpt2.txt')
small_jokes_prep_outpath = os.path.join('data', 'prep', 'small_jokes_gpt2.txt')
transcripts_prep_outpath = os.path.join('data', 'prep', 'transcripts_gpt2.txt')

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## Load data

### Jokes

In [2]:
data = pd.read_csv(qa_jokes_filepath)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38234 entries, 0 to 38233
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        38234 non-null  int64 
 1   Question  38234 non-null  object
 2   Answer    38234 non-null  object
dtypes: int64(1), object(2)
memory usage: 896.2+ KB


In [3]:
def fix_encoding(s):
    """Skip characters that can't be encoded by standard encoder."""
    return s.encode('latin1', 'ignore').decode('utf8', 'ignore')

START_DOC_TOKEN = '<|startoftext|>'
END_DOC_TOKEN = '<|endoftext|>'


text_corpus = ''
for _, question, answer in data.values:
    text_corpus += fix_encoding(f'{START_DOC_TOKEN}[QUESTION] {question}\n[ANSWER] {answer}\n{END_DOC_TOKEN}')
text_corpus += '\n\n'

with open(qa_jokes_prep_outpath, 'w') as out_file:
    out_file.write(text_corpus)

### Transcripts

In [4]:
full_transcripts = ''
# Load transcripts.
for file_path in glob(os.path.join(transcripts_path, '*')):
    with open(file_path, 'r', encoding='utf8') as in_file:
        full_transcripts += START_DOC_TOKEN + ''.join(in_file.read()) + END_DOC_TOKEN

# Save all them as dataset.
with open(transcripts_prep_outpath, 'w') as out_file:
    out_file.write(fix_encoding(fix_encoding(full_transcripts)))

### Short Jokes

In [ ]:
# TODO

## Train

In [75]:
python_path = r'C:\Users\kzvda\Anaconda3\envs\tabnine\python.exe'

block_size = 512
learning_rate = 1e-5
warmup_steps = 0
num_train_epochs = 1
train_batch_size = 1

os.system('set export CUDA_VISIBLE_DEVICES="0"')

os.system(f'start {python_path} run_language_modeling.py'
    + ' --output_dir=output'
    + ' --model_type=gpt2'
    + ' --model_name_or_path=gpt2'
    + ' --do_train'
    + f' --block_size={block_size}'
    + f' --train_data_file={transcripts_prep_outpath}'
    + f' --learning_rate={learning_rate}'
    + f' --warmup_steps={warmup_steps}'
    + f' --num_train_epochs={num_train_epochs}'
    + f' --per_gpu_train_batch_size={per_gpu_train_batch_size}'
#     ' --do_eval'
#     f'--eval_data_file=$TEST_FILE'
         )

0

In [79]:
import subprocess

myenv = os.environ.copy()
myenv['CUDA_VISIBLE_DEVICES'] = '0'
command = [
    f'{python_path} run_language_modeling.py',
    ' --output_dir=output',
    ' --model_type=gpt2',
    ' --model_name_or_path=gpt2',
    ' --do_train',
    f' --block_size={block_size}',
    f' --train_data_file={transcripts_prep_outpath}',
    f' --learning_rate={learning_rate}',
    f' --warmup_steps={warmup_steps}',
    f' --num_train_epochs={num_train_epochs}',
    f' --per_gpu_train_batch_size={per_gpu_train_batch_size}',
]

subprocess.check_call(command, env=myenv)

CalledProcessError: Command '['C:\\Users\\kzvda\\Anaconda3\\envs\\tabnine\\python.exe run_language_modeling.py', ' --output_dir=output', ' --model_type=gpt2', ' --model_name_or_path=gpt2', ' --do_train', ' --block_size=512', ' --train_data_file=data\\prep\\transcripts_gpt2.txt', ' --learning_rate=1e-05', ' --warmup_steps=0', ' --num_train_epochs=1', ' --per_gpu_train_batch_size=2']' returned non-zero exit status 1.

In [81]:
(f'start {python_path} run_language_modeling.py'
    + ' --output_dir=output'
    + ' --model_type=gpt2'
    + ' --model_name_or_path=gpt2'
    + ' --do_train'
    + f' --train_data_file={transcripts_prep_outpath}'
    + f' --learning_rate={learning_rate}'
    + f' --warmup_steps={warmup_steps}'
    + f' --num_train_epochs={num_train_epochs}'
    + f' --per_gpu_train_batch_size={1}')

'start C:\\Users\\kzvda\\Anaconda3\\envs\\tabnine\\python.exe run_language_modeling.py --output_dir=output --model_type=gpt2 --model_name_or_path=gpt2 --do_train --train_data_file=data\\prep\\transcripts_gpt2.txt --learning_rate=1e-05 --warmup_steps=0 --num_train_epochs=1 --per_gpu_train_batch_size=1'

In [ ]:
--k

In [ ]:
%env CUDA_VISIBLE_DEVICES='0'

env: CUDA_VISIBLE_DEVICES='0'


In [5]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=out_file_path,
              model_name='355M',
              steps=1000,
#               restore_from='fresh',
              restore_from='latest',
              run_name='run1',
              learning_rate=1e-5,
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint checkpoint\run1\model-500
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from checkpoint\run1\model-500


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.81s/it]


dataset has 1047992 tokens
Training...
Saving checkpoint\run1\model-500
[510 | 59.64] loss=2.27 avg=2.27
[520 | 78.28] loss=2.48 avg=2.38
[530 | 97.08] loss=2.26 avg=2.34
[540 | 116.03] loss=2.34 avg=2.34
[550 | 135.06] loss=2.51 avg=2.37
[560 | 153.80] loss=2.64 avg=2.42
[570 | 172.77] loss=2.26 avg=2.40
[580 | 191.54] loss=2.42 avg=2.40
[590 | 210.41] loss=2.38 avg=2.40
[600 | 229.22] loss=2.41 avg=2.40
======== SAMPLE 1 ========
 Because he was afraid of what would follow.

q: What do the two of them say when they go on a date?
a: They have fun, man

q: What do you have to do with the two of them?
a: You either have fun with me or my dick.

q: Did you hear about the two dudes who stole the eggs from the chickens??
a: They're still working on justice.

q: Why are there no lesbians at a strip club?
a: A club with only one option. 

q: Why do I always sit in the rain?
a: Because I can't get out.

q: What is its name, the chicken or the egg?
a: The chicken.

q: Why was the dog so scared

[890 | 852.58] loss=2.03 avg=2.31
[900 | 871.31] loss=2.23 avg=2.31
[910 | 890.11] loss=2.31 avg=2.31
[920 | 908.76] loss=2.28 avg=2.31
[930 | 927.57] loss=2.22 avg=2.31
[940 | 946.38] loss=2.37 avg=2.31
[950 | 965.37] loss=2.34 avg=2.31
[960 | 984.39] loss=2.14 avg=2.30
[970 | 1003.18] loss=2.24 avg=2.30
[980 | 1021.91] loss=2.32 avg=2.30
[990 | 1040.60] loss=2.31 avg=2.30
[1000 | 1059.52] loss=2.30 avg=2.30
Saving checkpoint\run1\model-1000
Instructions for updating:
Use standard file APIs to delete files with this prefix.
======== SAMPLE 1 ========
 you have any questions or comments??? Thanks so much!!!<|endoftext|>Punjabi poet Sanjeev Saxena today launched an online charity drive to benefit the women of Bengaluru.

"I have spent many hours thinking and writing about women's empowerment, but the task is daunting. It is one thing to give my money to Women's Commission of India (WCOI), another to give Rs 10 lakh to some lady's family, that way the WCOI would get enough money and the 

[1210 | 1560.14] loss=2.26 avg=2.32
[1220 | 1579.08] loss=2.48 avg=2.33
[1230 | 1597.66] loss=2.45 avg=2.33
[1240 | 1616.48] loss=2.24 avg=2.33
[1250 | 1635.45] loss=2.45 avg=2.33
[1260 | 1654.31] loss=2.22 avg=2.33
[1270 | 1673.03] loss=2.30 avg=2.33
[1280 | 1691.80] loss=2.40 avg=2.33
[1290 | 1710.61] loss=2.21 avg=2.33
[1300 | 1729.53] loss=2.43 avg=2.33
[1310 | 1748.31] loss=2.40 avg=2.33
[1320 | 1766.98] loss=2.47 avg=2.33
[1330 | 1785.68] loss=2.41 avg=2.33
[1340 | 1804.63] loss=2.31 avg=2.33
[1350 | 1823.45] loss=2.36 avg=2.33
[1360 | 1842.24] loss=2.26 avg=2.33
[1370 | 1861.03] loss=2.03 avg=2.33
[1380 | 1879.84] loss=2.41 avg=2.33
[1390 | 1898.75] loss=2.44 avg=2.33
[1400 | 1917.65] loss=2.36 avg=2.33
======== SAMPLE 1 ========
 than two months without having sex?
a: A pregnant woman.

q: What does a lesbian lawyer do when she's not around her partner?
a: Call an Uber.

q: Why can't you hear the croaks in a bird cage?
a: Its a birdcage.

q: Why is that black guy's penis so big

In [5]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint\run1\model-1500
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from checkpoint\run1\model-1500


In [18]:
gpt2.generate(sess, run_name='run1',
              temperature=0.7,
              batch_size=1,
              nsamples=20,
              top_k=40,
#               top_p=0.8,
              length=50,
              prefix="Q:",
              truncate="\n\n",
             )

Q: What is the difference between a cat and a dog?
a: One has a whisker and the other is a fur coat.
Q: What do you call a bald man with a gun?
a: A man with a gun
Q: What do you call a girl with a belly button?
a: A belly button
Q: What do you call it when a woman with a baby in her mouth goes through a door?
a: An anal-bod
Q: What does a frog say when it sees a wall?
a: "You have a wall, but no floor!"
Q: What do you call a black guy that can't stand up straight?
a: A stoner.
Q: What do you call it when a woman has a baby?
a: Infertility
Q: What do you call it when you get a pair of pants you don't like?
a: Pants-down.
Q: How do you tell if a person is a vegetarian?
a: You don't. 
Q: What do you call a black guy who is an atheist?
a: A black atheist. 
Q: What did the doctor say to the terminally ill man?
a: "Keep your legs up."
Q: What did the chicken say to the cow?
a: Chicken, come on, I'm a chicken!
Q: What's a woman's worst enemy?
a: Her husband
Q: What's a good way to kill a man